In [1]:
# imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import pyautogui

import time

In [2]:
#start webdriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 3.81MB/s]


In [3]:
driver.get("https://www.google.com")

search = driver.find_element(By.CLASS_NAME, 'gLFyf')
search.send_keys('snake game')
search.send_keys(Keys.RETURN)


In [4]:
play = driver.find_element(By.CLASS_NAME, "XlTvtc")

play.click()
driver.implicitly_wait(4)




In [5]:
### TODO: SET TURTLE MODE

settings = driver.find_element(By.XPATH, "//div[@jsname='iyH4Cb']")
settings.click()
driver.implicitly_wait(3)

turtle = driver.find_element(By.XPATH, "//img[@src='//www.google.com/logos/fnbx/snake_arcade/v3/speed_02.png']")
actions = ActionChains(driver)
actions.move_to_element(turtle).click().perform()

driver.implicitly_wait(2)

In [6]:
play_2 = driver.find_element(By.XPATH, "//div[@jsname='NSjDf']")
play_2.click()


In [7]:
# DEFINE GAME WINDOW

game = driver.find_element(By.XPATH, '//body')
# driver.maximize_window()
driver.implicitly_wait(3)

#find corners of game window
top_left_x, top_left_y, top_left_width, top_left_height = pyautogui.locateOnScreen('top-left.jpg', confidence = 0.98)
top_right_x, top_right_y, top_right_width, top_right_height = pyautogui.locateOnScreen('top-right.jpg', confidence = 0.98)
bottom_left_x, bottom_left_y, bottom_left_width, bottom_left_height = pyautogui.locateOnScreen('bottom-left.jpg', confidence = 0.98)
bottom_right_x, bottom_right_y, bottom_right_width, bottom_right_height = pyautogui.locateOnScreen('bottom-right.jpg', confidence = 0.98)

bottom_left_x += 6
bottom_left_y += 19

bottom_right_x += 19
bottom_right_y += 20

top_left_x += 7
top_left_y += 6

top_right_x += 20
top_right_y += 7

print('c1', top_left_x, top_left_y)
print('c2', top_right_x, top_right_y)
print('c3', bottom_left_x, bottom_left_y)
print('c4', bottom_right_x, bottom_right_y)

# all of these are roughly square: 26x27 or 27x26 pixel images
# height = (bottom_right_y + bottom_left_y)//2 - (top_right_y + top_left_y)//2 - top_left_height #subtract this from both estimates
# width = (bottom_right_x + top_right_x)//2 - (bottom_left_x + top_left_x)//2 - top_left_width  #because a little extra padding on images

height = bottom_left_y - top_left_y
width = top_right_x - top_left_x

boxh = (height / 15)
boxw = (width / 17)
print(boxw, boxh)

LEFT_BOX_X = int(top_left_x + boxw/2.25) #divided by 3 so that is closer to edge and extened mouth doesn't trigger movement
TOP_BOX_Y = int(top_left_y + boxh/2.25)

RIGHT_BOX_X = int(top_right_x - boxw/2.25)
BOTTOM_BOX_Y = int(bottom_left_y - boxw/2.25)

#measurements are imperfect, but good enough if we go from top/bottom for upper/lower rows
### NEED TO REFACTOR, ONLY WORKS WITH CERTAIN WINDOW SIZES
# THIS CAN MISS THE APPLE (not necessary for this program though)
def box_pixel(box_x, box_y):
    if box_x < 9:
        x_pixel = LEFT_BOX_X + boxw*(box_x - 1)
    else:
        x_pixel = RIGHT_BOX_X - boxw*(17-box_x)
    
    if box_y < 9:
        y_pixel = TOP_BOX_Y + boxh*(box_y - 1)
    else:
        y_pixel = BOTTOM_BOX_Y - boxh*(15-box_y)
    return int(x_pixel), int(y_pixel)

snake_x, snake_y = box_pixel(3, 8) #starting snake location
SNAKE_RGB = pyautogui.pixel(snake_x, snake_y) #snake color for reference
print(SNAKE_RGB)

c1 1112 298
c2 1790 298
c3 1112 895
c4 1789 896
39.88235294117647 39.8
(80, 116, 234)


In [8]:
#waits for snake to reach a point and triggers 1-2 key movements
def trigger(target_x, target_y, key1, key2='empty'):
    pix_x, pix_y = box_pixel(target_x, target_y)
    
    tail_clear = True
    if pyautogui.pixelMatchesColor(pix_x, pix_y, SNAKE_RGB, tolerance = 30):
        tail_clear = False
        
    while tail_clear == False:
        if not (pyautogui.pixelMatchesColor(pix_x, pix_y, SNAKE_RGB, tolerance=30)):
            tail_clear = True
            
    fired = False
    while fired == False:
        if pyautogui.pixelMatchesColor(pix_x, pix_y, SNAKE_RGB, tolerance = 30):##need tolerance for changes in mouth when apple is close
            game.send_keys(key1)
            if key2 != 'empty':
                game.send_keys(key2)
            fired = True

In [9]:
#START SNAKE
game.send_keys(Keys.ARROW_RIGHT)

for row in range(7):
    if row % 2 == 0:
        trigger(17, 8 + row, Keys.ARROW_DOWN, Keys.ARROW_LEFT)
    else:
        trigger(2, 8 + row, Keys.ARROW_DOWN, Keys.ARROW_RIGHT)

trigger(1, 15, Keys.ARROW_UP)
trigger(1, 1, Keys.ARROW_RIGHT)

runs = 0
while True:
    for row in range(14):
        if (row + runs) % 2 == 0:
            trigger(16, row+1, Keys.ARROW_DOWN, Keys.ARROW_LEFT)
        else:
            trigger(2, row+1, Keys.ARROW_DOWN, Keys.ARROW_RIGHT)
    if runs % 2 == 0:
        trigger(17, 15, Keys.ARROW_UP)
        trigger(17, 1, Keys.ARROW_LEFT)
    else:
        trigger(1, 15, Keys.ARROW_UP)
        trigger(1, 1, Keys.ARROW_RIGHT)
    runs += 1

    

KeyboardInterrupt: 